## Import Libraries

In [1]:
import matplotlib.pyplot as plt
import urllib
from bs4 import BeautifulSoup
from selenium import webdriver
import re
import os,sys, shutil
import time
from datetime import date
try:
    import cPickle as pickle
except:
    import pickle
import pprint
from collections import deque
from shutil import copyfile
import random
import glob
# Import the required modules
import cv2, os
import numpy as np
from PIL import Image
from sklearn.cross_validation import KFold
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
# Logistic Regression
from sklearn import datasets
from sklearn import metrics

In [15]:
pkl_fl = open("linkedin_profiles.pickle","rb")
my_original_list=pickle.load(pkl_fl) # errors out here
pkl_fl.close()

## label the Images and store into respective folders

In [16]:
directory = "Male"
    
if not os.path.exists(directory):
    os.makedirs(directory)

directory1 = "Female"

if not os.path.exists(directory1):
    os.makedirs(directory1)    

directory2 = "Label_Images_Gender"

if os.path.exists(directory2):
    shutil.rmtree(directory2)
    os.makedirs(directory2)     
else:
    os.makedirs(directory2)     
    
fileList = glob.glob("./Images/*.*")

for id,fp in enumerate(fileList):
    filename, file_extension = os.path.splitext(fp)
    uid = filename.split('/')[-1]
    #print fp
    for prof in my_original_list:
        if prof['User_ID'] == uid:
            new_file_extension = prof['Gender']
            new_file_extension = new_file_extension.title()
            #os.rename(fp, filename+"."+new_file_extension)
            copyfile(filename+".jpg", './Label_Images_Gender/'+ uid + '.' + str(id) + "."+new_file_extension +'.jpg')
            copyfile(filename+".jpg", new_file_extension +'/' + uid + ".jpg")

In [6]:
# For face detection we will use the Haar Cascade provided by OpenCV.
cascadePath = "haarcascade_frontalface_default.xml"
faceCascade = cv2.CascadeClassifier(cascadePath)

# For face recognition we will the the LBPH Face Recognizer 
recognizer = cv2.createLBPHFaceRecognizer()

In [8]:
def get_images_and_labels(path):
    # Append all the absolute image paths in a list image_paths
    
    image_paths = [os.path.join(path, f) for f in os.listdir(path)]
    # images will contains face images
    images = []
    # labels will contains the label that is assigned to the image
    labels = []
    #gender will contains 1 or 0 indecating male or female
    gender =[]
    
    for image_path in image_paths:
        # Read the image and convert to grayscale
        try:
            image_pil = Image.open(image_path).convert('L')
            # Convert the image format into numpy array
            image = np.array(image_pil, 'uint8')
            # Get the label of the image
        except:
            pass
        
        nbr = int(os.path.split(image_path)[1].split(".")[1])
        gender_current = os.path.split(image_path)[1].split(".")[2]
        print nbr
        
        # Detect the face in the image
        faces = faceCascade.detectMultiScale(image)
        # If face is detected, append the face to images and the label to labels
        try:
            for (x, y, w, h) in faces:

                ref_image = image[y: y + h, x: x + w]
                resized = cv2.resize(ref_image, (100, 100), interpolation = cv2.INTER_AREA)
                images.append(resized)
                labels.append(nbr)

                if gender_current == 'Male':
                    gender.append(1)
                else:
                    gender.append(0)

                cv2.imshow("Adding faces to traning set...", resized)
                cv2.waitKey(10)
        except:
            pass
    # return the images list and labels list
    print "lables"
    print labels
    print "gender_current"
    print gender
    
    return images, labels, gender

In [17]:
images, labels, gender = get_images_and_labels('Label_Images_Gender')

cv2.destroyAllWindows()

426
1982
1484
957
1715
1460
1449
306
451
234
15
134
928
955
380
1061
46
1838
733
952
1563
356
1008
657
1791
2002
1249
177
823
987
1372
279
1519
779
84
1271
1640
829
723
1570
751
1644
1270
59
1745
1213
1496
1200
1987
1827
1979
2126
1461
261
788
1552
217
193
1465
197
729
254
1892
1111
1114
2005
1274
846
1230
156
1579
1291
1929
1310
710
1034
1371
1087
1964
627
1104
1684
1464
773
317
1218
1152
1918
1190
1991
792
994
871
1620
1762
690
1948
646
2065
152
1253
1812
2053
1001
1650
162
1788
1691
1782
2054
1229
1340
1597
287
809
2082
2051
362
1572
381
1915
446
289
89
32
1947
1676
1054
1888
1687
330
1389
383
1080
755
1712
1885
1471
458
1226
230
1196
2015
576
1285
2072
1822
929
1808
608
940
1968
2016
1513
1499
1612
54
2018
83
335
1248
1667
1662
516
1024
1074
1158
246
1721
2076
248
111
609
1304
340
698
1124
1411
1843
24
1719
194
860
1421
370
554
1237
605
1052
915
216
519
1554
105
27
971
1515
1015
1984
2136
1305
48
373
2070
1038
1185
2119
1101
17
423
1683
1335
1714
1321
2008
1342
122
1273
599
1665
88

## Making the Dataset 

In [19]:
res_images = []
res_gender = []

for img in images:
    res_images.append(img.reshape(1,10000))    # (100*100 image size)

for gen in gender:
    res_gender.append(np.array(gen).reshape(1,))
    
res_images = np.concatenate(res_images, axis=0)
res_gender = np.concatenate(res_gender,axis=0)

In [20]:
res_images.shape

(1879, 10000)

In [21]:
res_gender.shape

(1879,)

In [22]:
X,y = res_images,res_gender

## Test the Model for Age

In [25]:
kf = KFold(X.shape[0], n_folds=10, shuffle=True)
accuracy = 0.0
precision = [0, 0]
recall = [0, 0]
f_measure = [0, 0]

lr = LogisticRegression()
inbuilt_accuracy_score = 0.0

for test, train in kf:
    X_Train, X_Test = X[train], X[test]
    Y_Train, Y_Test = y[train], y[test]

    lr.fit(X_Train, Y_Train)

    

    pred = lr.predict(X_Test)

    inbuilt_accuracy_score += accuracy_score(Y_Test, (pred))

#precision = map(lambda x: x / 10, precision)
#recall = map(lambda x: x / 10, recall)
#f_measure = map(lambda x: x / 10, f_measure)
#accuracy /= 10
#print "Confusion Matrix : \n\n", cm

inbuilt_accuracy_score /= 10

#print "*" * 100
#print "\t\t\t Inbuilt Method"

print "\nAccuracy from inbuilt Method: ", inbuilt_accuracy_score
print(metrics.classification_report(Y_Test, pred))
print(metrics.confusion_matrix(Y_Test, pred))



Accuracy from inbuilt Method:  0.689610271595
             precision    recall  f1-score   support

          0       0.63      0.73      0.68       752
          1       0.75      0.66      0.70       940

avg / total       0.70      0.69      0.69      1692

[[549 203]
 [321 619]]
